# Exploratory Data Analysis
For this example, I downloaded several data sets on health, economy, and education from [Gapminder](http://www.gapminder.org/data/). The data are in excel files, and all files are located in the subdirectory "data".

In [ ]:
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from __future__ import print_function # Python 2 users should uncomment this statement

In [ ]:
# First read in each data set into a separate dataframe
# The dataframes will be stored in a dictionary with the filename as the key
data = {}
datadir = os.path.join("data", "gapminder")
for fname in os.listdir(datadir):
    dfname = fname.rstrip(".xls")
    data[dfname] = pd.read_excel(os.path.join(datadir, fname), index_col=0)

In [ ]:
# Let's see what we have
# To display multiple lines of output, we need to use the print function.
print("dataframes:", data.keys())
for k,v in data.items():
    print("name:", k)
    print("columns:", v.columns)
    print("index (row name):", v.index)

    
# Scroll through the output below and you will see that each data set has a column for each year of data
# and the rows correspond to countries.

## Clean
A good data cleaning step here is to check that the country names are consistant and to make the dataframe names shorter, without spaces, because in future steps these will become column names.

In [ ]:
# Examine the names
data.keys()

In [ ]:
# Make a dictionary that maps old names to new names.
# Note that renaming would have been more effiently done at time of download, but this way we have a record of it.
namemap = {'GDPpercapitaconstant2000US':'gdp_per_capita',
            'indicatorwdigdp_percapita_growth':'gdp_growth_per_capita',
            'indicator life_expectancy_at_birth':'life_expectancy',
            'indicator_government share of total health spending':'health_spending_govt_share',
            'indicator_population density (per square km)':'population_density',
            'indicator_per capita government expenditure on health at average exchange rate (us$)':'health_spending_govt_percap',
            'Internet user per 100':'internet_use'}
for k,v in namemap.items():
    data[v] = data.pop(k)

### Exercise
Check the country names to verify they are consistant among all the dataframe.

## Merge

I want to find the most recent year with data in all data sets, then merge the data for just that year from all the data sets into one dataframe. 

Some of the years are represented as strings, and others as integers. For sucessful operations, we need to make this consistant. 

In [ ]:
for v in data.values():
    v.columns = [int(c) for c in v.columns]

In [ ]:
years = [set(v.columns) for v in data.values()]
overlap = years[0]
for y in years[1:]:
    overlap = overlap.intersection(y)
max(list(overlap))

In [ ]:
# Create a dataframe with 2005 data from all the datasets, using the dataframe key as the column name
# Pandas has a merge function for two data frames, but I think in this instance it is easier to pull out the desired 
# columns and make a new one that way.
data2010 = pd.DataFrame(dict([(k, v[2010]) for k,v in data.items()]))

In [ ]:
data2010.describe()

In [ ]:
# drop nans because graphing package seaborn doesn't handle them.
data2010_nonan = data2010.dropna()
data2010_nonan.describe()

In [ ]:
sns.set()
sns.pairplot(data2010_nonan)

### Exercise
Identify non-linear relationships from the above matrix of scatter plots. Make new features by transforming variables that create linear relationships. Make scatter plots of the new features to verify the linearity. 

### Exercise
View the pandas documentation on the [merge](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) function. Use it to merge two of the available dataframes. Note that due to a conflict in the column names, you should first rename the columns

## Bin variables by percentile

In [ ]:
# pandas qcut will give quartiles, or any number of equally sized bins
quartlabels = ["25th percentile", "50th percentile", "75th percentile", "100th percentile"]
data2010["life_expectancy_quartiles"] = pd.qcut(data2010["life_expectancy"], 4, labels=quartlabels)
data2010["population_density_quartiles"] = pd.qcut(data2010["population_density"], 4, labels=quartlabels)

tenthlabels = ["{:n}th percentile".format(p) for p in range(10,101,10)]
data2010["gdp_tenths"] = pd.qcut(data2010["gdp_per_capita"], 10, labels=tenthlabels)

In [ ]:
# create some contingency tables
density_gdp_table = pd.crosstab(data2010["population_density_quartiles"],data2010["gdp_tenths"])
life_gdp_table = pd.crosstab(data2010["life_expectancy_quartiles"],data2010["gdp_tenths"])
life_density_table = pd.crosstab(data2010["life_expectancy_quartiles"], data2010["population_density_quartiles"])

In [ ]:
density_gdp_table

In [ ]:
life_gdp_table

In [ ]:
life_density_table

In [ ]:
# make a mosiac plot. Note, this fails if any value is zero. Also axes labels are not working properly.
from statsmodels.graphics.mosaicplot import mosaic as statsmodels_mosaic
plottable = life_density_table.stack() # the mosiac plot needs a particular data structure
statsmodels_mosaic(plottable, axes_label=True)
plt.xlabel(plottable.index.names[0], verticalalignment="bottom")
plt.ylabel(plottable.index.names[1])
plt.show()

In [ ]:
# Scatter plot example in lecture
plt.scatter(data2010["health_spending_govt_percap"], data2010["life_expectancy"])
plt.xlabel("health_spending_govt_percap")
plt.ylabel("life_expectancy")
plt.title("Scatter Plot")